In [267]:
import pandas as pd

In [268]:
data_frame = pd.read_csv("nba_games.csv", index_col = 0)

In [269]:
data_frame = data_frame.sort_values("date")

In [270]:
data_frame = data_frame.reset_index(drop=True)

In [271]:
del data_frame["mp.1"]
del data_frame["mp_opp.1"]
del data_frame["index_opp"]

In [272]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

data_frame = data_frame.groupby("team", group_keys=False).apply(add_target)

C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\531162783.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\531162783.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\531162783.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining al

In [273]:
data_frame["target"][pd.isnull(data_frame["target"])] = 2

C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\2906054394.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data_frame["target"][pd.isnull(data_frame["target"])] = 2
C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\2906054394.py:1: Set

In [274]:
data_frame["target"] = data_frame["target"].astype(int, errors="ignore")

In [275]:
data_frame["won"].value_counts()

won
False    8886
True     8886
Name: count, dtype: int64

In [276]:
data_frame["target"].value_counts()

target
1    8872
0    8870
2      30
Name: count, dtype: int64

In [277]:
nulls = pd.isnull(data_frame).sum()

In [278]:
valid_columns = data_frame.columns[~data_frame.columns.isin(nulls[nulls > 0].index)]

In [279]:
data_frame = data_frame[valid_columns].copy()

In [280]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import RidgeClassifier


ridge= RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)
gradient_booster = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

In [281]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]

In [282]:
selected_columns = data_frame.columns[~data_frame.columns.isin(removed_columns)]

In [283]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data_frame[selected_columns] = scaler.fit_transform(data_frame[selected_columns])

In [284]:
predictors = selected_columns

In [285]:
def backtest(data, model, predictors, start=2, step=1):
    predictions = []
    seasons = sorted(data["season"].unique())
    for i in range(start, len(seasons), step):
        season = seasons[i]
        training = data[data["season"] < season]
        test = data[data["season"] == season]
        model.fit(training[predictors], training["target"])
        prediction = model.predict(test[predictors])
        prediction = pd.Series(prediction, index=test.index)
        combined = pd.concat([test["target"], prediction], axis=1)
        combined.columns = ["actual", "prediction"]
        predictions.append(combined)
    return pd.concat(predictions)

In [286]:
predictions = backtest(data_frame, ridge, predictors)

In [287]:
from sklearn.metrics import accuracy_score


predictions = predictions[predictions["actual"] != 2]
accuracy_score(predictions["actual"], predictions["prediction"])

0.5330611591418508

In [288]:
data_frame.groupby("home").apply(lambda x: x[x["won"]==1].shape[0] /x.shape[0])

C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\131860665.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_frame.groupby("home").apply(lambda x: x[x["won"]==1].shape[0] /x.shape[0])


home
-1.0    0.428314
 1.0    0.571686
dtype: float64

In [289]:
data_frame_rolling = data_frame[list(selected_columns) + ["won", "team", "season"]]

In [290]:
import numpy as np

In [291]:
data_frame_rolling['won'] = data_frame_rolling['won'].astype(int)
def find_team_averages(team):
    numeric_team = team.select_dtypes(include=[np.number])
    rolling = numeric_team.rolling(10).mean()
    return rolling


data_frame_rolling = data_frame_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)

C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\418993166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame_rolling['won'] = data_frame_rolling['won'].astype(int)
C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\418993166.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_frame_rolling = data_frame_rolling.groupby(["team", "season"], group_keys=False).apply(find_team_averages)


In [292]:
rolling_columns = [f"{column}_10" for column in data_frame_rolling.columns]
data_frame_rolling.columns = rolling_columns

data_frame = pd.concat([data_frame, data_frame_rolling], axis=1)

In [293]:
data_frame = data_frame.dropna()

In [294]:
def shift_column(team, column_name):
    next_column = team[column_name].shift(-1)
    return next_column

def add_column(data_frame, column_name):
    return data_frame.groupby("team", group_keys=False).apply(lambda x: shift_column(x, column_name))

data_frame["home_next"] = add_column(data_frame, "home")
data_frame["team_opp_next"] = add_column(data_frame, "team_opp")
data_frame["date_next"] = add_column(data_frame, "date")

C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\242433026.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return data_frame.groupby("team", group_keys=False).apply(lambda x: shift_column(x, column_name))
C:\Users\sidhu\AppData\Local\Temp\ipykernel_16500\242433026.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return data_frame.groupby("team", group_keys=False).apply(lambda x: shift_column(x, colu

In [295]:
data_frame = data_frame.copy()

In [296]:
full = data_frame.merge(data_frame[rolling_columns + ["team_opp_next", "date_next", "team"]], 
        left_on = ["team", "date_next"], right_on=["team_opp_next", "date_next"])

In [297]:
full[["team_x", "team_opp_next_x", "team_y", "team_opp_next_y", "date_next"]]

,team_x,team_opp_next_x,team_y,team_opp_next_y,date_next
0,SAC,TOR,TOR,SAC,2015-11-15
1,TOR,SAC,SAC,TOR,2015-11-15
2,CLE,DET,DET,CLE,2015-11-17
3,GSW,TOR,TOR,GSW,2015-11-17
4,DEN,NOP,NOP,DEN,2015-11-17
...,...,...,...,...,...
15769,BOS,GSW,GSW,BOS,2022-06-10
15770,GSW,BOS,BOS,GSW,2022-06-13
15771,BOS,GSW,GSW,BOS,2022-06-13
15772,GSW,BOS,BOS,GSW,2022-06-16


In [298]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [299]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [300]:
predictors = selected_columns

In [301]:
predictions = backtest(full, ridge, predictors)

In [302]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

accuracy = accuracy_score(predictions["actual"], predictions["prediction"])
print(f"Accuracy: {accuracy*100}%")

print("\n")

print("Classification Report:")
print(classification_report(predictions["actual"], predictions["prediction"]))

print("\n")

print("Confusion Matrix:")
print(confusion_matrix(predictions["actual"], predictions["prediction"]))



Accuracy: 61.124628277912954%


Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.61      0.61      5553
           1       0.61      0.61      0.61      5544

    accuracy                           0.61     11097
   macro avg       0.61      0.61      0.61     11097
weighted avg       0.61      0.61      0.61     11097



Confusion Matrix:
[[3412 2141]
 [2173 3371]]
